In [80]:

import warnings
import pandas as pd
from typing import List
warnings.filterwarnings("ignore")
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    PromptTemplate, ChatPromptTemplate
)
from langchain_community.utils.openai_functions import (
    convert_pydantic_to_openai_function,
)
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [5]:
model_name = 'gpt-4-1106-preview'

In [148]:
class Answer(BaseModel):
    """Unique Human-like survey answers told by each person. Each person should have their own unique answers"""

    answer1: str = Field(description="This is the human like answer to Question1.")
    answer2: str = Field(description="This is the human like answer to Question2.")
    answer3: str = Field(description="This is the human like answer to Question3.")
    answer4: str = Field(description="This is the human like answer to Question4.")
    traits: str = Field(description="This is the trait of the person.")

In [149]:
class Answers(BaseModel):
    """Human-like survery answers to tell user."""

    answer: List[Answer]

In [150]:
openai_functions = [convert_pydantic_to_openai_function(Answers)]

In [151]:
parser = JsonKeyOutputFunctionsParser(key_name="answer")

In [152]:
llm = ChatOpenAI(
    temperature=1,
    model_name=model_name
).bind(functions=openai_functions)

In [176]:
system_message = """
You are a survey answering bot that generates answers like a survey when questions are asked. The answer should be made as if you are a human. 
Give answers assuming you are a new human with different lifestyles while giving answers.
Give answers with new thoughts, new ideas, new moods, you can also choose to answer very rudely, but the main idea is to be random, do not try to repeat same answers since humans have different answers with different answer style.
While giving answers, be as creative as possible and try to deviate your answers as much as possible from previous answers.
In every answer, change styles of answers, change average sentence lengths of answer, change fk_grade_level of sentences of answer. Make it different from previous answers. But also make sure it is the answer given by a human. So, don't make it seem like it is AI generated. Add both simple and fancy words.
In 1 answer, give your answers assuming you are having a worse life, in another answer, give your answers assuming you are having best life. like this, keep on changing the lifestyle of human that you are.
Avoid same repeated answers as much as possible.
Do no repeat same pattern in each answer. Give short answers sometimes and sometimes long answers, be random.
Since, human can give both positive and negative answers, you should follow the same principles.
Your answers should be descriptive just like human answers.
If the output asks for a monetary output give just the dollar amount and no text before or after it unless prompted to do so. So for example if the answer is $40 just give $40.
If the output asks for a decimal output give just the value and no text before or after it unless prompted to do so. So for example if the answer is 40.5 just give 40.5.
If you are asked a question like where do you live? Be sure to just answer the place that you live. No text before or after it is needed unless prompted to do so.
If the answer is asking for a percentage output just give the value. So if the answer is 45% just give 45%. Do not give any text before or after that unless prompted to do so.
"""

In [177]:

prompt = ChatPromptTemplate.from_messages(
    [("system", system_message), ("user", "{input}")]
)

In [178]:
chain = prompt | llm | parser

In [179]:
audience = 'Baby Boomers in the United States'
traits_and_counts = [
    ('Men being currently employed', 5),
    ('Women being currently employed', 5)
]

questions = [
    'How happy are you with your life on a scale of 1-5 where 1 is very unhappy and 5 is very happy? (#)',
   'What are the top few activities that make you happiest?',
    'Why do these activities make you happy?',
    'Where do you work? From home, the office, or a combination of both.'
    
]



In [180]:

traits1 = traits_and_counts[0][0]
counts1 = traits_and_counts[0][1]
traits2 = traits_and_counts[1][0]
counts2 = traits_and_counts[1][1]
total_surveyed = 0
for dta in traits_and_counts:
    total_surveyed += dta[1]
print("Total surveyed: ", total_surveyed)


Total surveyed:  10


In [181]:

final_data = []

input_message = f"""
                Give survey answers from {total_surveyed} people. 
                All of the surveyed people are {audience}
                {counts1} of the surveyed people have this trait: {traits1}
                Remaining {counts2} of the surveyed people have this trait: {traits2}
                Question1: {questions[0]}
                Question2: {questions[1]}
                Question3: {questions[2]}
                Question4: {questions[3]}
                """
responses = chain.invoke({"input": input_message})
for ind, data in enumerate(responses):
    person_number = ind+1
    final_data.append({'Traits': data['traits'], 'Person Number': person_number, f'{questions[0]}': data['answer1'], f'{questions[1]}': data['answer2'],
                        f'{questions[2]}': data['answer3'], f'{questions[3]}': data['answer4']})
print(responses)

[{'answer1': '5', 'answer2': 'Gardening, reading, and spending time with grandkids.', 'answer3': 'They bring me peace and joy, especially seeing things grow and family bonding.', 'answer4': 'From home', 'traits': 'Men being currently employed'}, {'answer1': '3', 'answer2': 'Golfing, woodworking, and watching old movies.', 'answer3': "It's my me-time, gets me outside, and I appreciate the craftsmanship and nostalgia.", 'answer4': 'The office', 'traits': 'Men being currently employed'}, {'answer1': '4', 'answer2': 'Fishing, volunteering, and having barbecues.', 'answer3': "I feel connected to nature, it's fulfilling to give back, and who doesn't love good food with friends?", 'answer4': 'Combination of both', 'traits': 'Men being currently employed'}, {'answer1': '2', 'answer2': 'Collecting stamps, playing piano, and chess.', 'answer3': "They're intellectually stimulating and provide a sense of accomplishment.", 'answer4': 'The office', 'traits': 'Men being currently employed'}, {'answer

In [182]:
df = pd.DataFrame.from_dict(final_data)

In [183]:
df

,Traits,Person Number,How happy are you with your life on a scale of 1-5 where 1 is very unhappy and 5 is very happy? (#),What are the top few activities that make you happiest?,Why do these activities make you happy?,"Where do you work? From home, the office, or a combination of both."
0,Men being currently employed,1,5,"Gardening, reading, and spending time with gra...","They bring me peace and joy, especially seeing...",From home
1,Men being currently employed,2,3,"Golfing, woodworking, and watching old movies.","It's my me-time, gets me outside, and I apprec...",The office
2,Men being currently employed,3,4,"Fishing, volunteering, and having barbecues.","I feel connected to nature, it's fulfilling to...",Combination of both
3,Men being currently employed,4,2,"Collecting stamps, playing piano, and chess.",They're intellectually stimulating and provide...,The office
4,Men being currently employed,5,4,"Hiking, painting, and playing bridge.","I love the physical challenge, expressing crea...",From home
5,Women being currently employed,6,4,"Yoga, book club, and gardening.","They help me maintain balance, stimulate my mi...",The office
6,Women being currently employed,7,5,"Traveling, cooking exotic meals, and dancing.","I enjoy cultural experiences, creating tasty d...",From home
7,Women being currently employed,8,3,"Knitting, watching plays, and teaching my gran...",These activities are tranquil and provide a wa...,Combination of both
8,Women being currently employed,9,4,"Volunteering at the animal shelter, running, a...","I adore animals, cherish the runner's high, an...",The office
9,Women being currently employed,10,3,"Jazzercise, scrapbooking, and wine tasting.","They're fun, creative, and offer a chance to s...",From home


In [184]:
df.to_csv('surveyResults6.csv', index=False)